# Data Collection

<hr>

Get an understanding of how [this](https://github.com/lmeazzini/youtube-recommender/blob/master/src/data_science/search_data_collection.py) works.

In [1]:
import time
import requests
import yaml

This isn't working :(

In [4]:
# open the .yaml file for parameters
with open('../parameters.yaml') as f:
    params = yaml.load(f, Loader=yaml.FullLoader)

# gets the parametesr from the .yaml file
queries = params['queries']
urls = params['urls']
number_of_pages = params['number_of_pages']

# Collecting data from youtube pages
for query in queries:
    for page in range(1, number_of_pages+1):
        url = urls.format(query=query, page=page)
        response = requests.get(url)

        with open('../../pages/{}_{}.html'.format(query,
                                                  page), 'w+') as output:
            output.write(response.text)

        time.sleep(1)

FileNotFoundError: [Errno 2] No such file or directory: '../../pages/folk+rock_1.html'

<hr>

New tutorial [here](https://www.youtube.com/watch?v=th5_9woFJmk). hehe

Load API key from the txt file. (There's probably a better way to do this, but oh well!)

In [3]:
API_KEY = ""

with open("../YouTube API Key.txt") as f:
    for line in f:
        API_KEY = line

Reference: https://github.com/googleapis/google-api-python-client  
Another reference: https://googleapis.github.io/google-api-python-client/docs/dyn/youtube_v3.html

## Create the YouTube Service object

Get the YouTube service to let us work with the APIs

In [4]:
# Allows us to connect to the YouTube API service
from googleapiclient.discovery import build

# the service
youtube_service = build('youtube', 'v3', developerKey=API_KEY)

Simple API request as a test (from the tutorial video)

In [5]:
request = youtube_service.channels().list(
            part='statistics',
            forUsername='schafer5'
        )

In [5]:
response = request.execute()

In [6]:
print(response)

{'kind': 'youtube#channelListResponse', 'etag': 'lXXlwIGiu-wf7INz1lLPOZIg9Bo', 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5}, 'items': [{'kind': 'youtube#channel', 'etag': '9ZvbmSCHow8TQjfuamaR6SnAnrA', 'id': 'UCCezIgC97PvUuR4_gbFUs5g', 'statistics': {'viewCount': '53160693', 'subscriberCount': '729000', 'hiddenSubscriberCount': False, 'videoCount': '230'}}]}


:)!

## List YouTube videos based off search keywords

*Now let's figure out how to search for YouTube music videos based on keywords.*

In [42]:
search_terms = 'xKito'
MUSIC_CATEGORY_ID = 10

In [43]:
MUSIC_CATEGORY_ID = 10

request = youtube_service.search().list(
            part='snippet',
            maxResults=50,
            q=search_terms,
            type='video',
            videoCategoryId = MUSIC_CATEGORY_ID,
            )

In [44]:
response = request.execute()

In [9]:
print(response)

{'kind': 'youtube#searchListResponse', 'etag': 'Opf4m43SnG1mhHpgQffTxtmYXnA', 'nextPageToken': 'CDIQAA', 'regionCode': 'US', 'pageInfo': {'totalResults': 144, 'resultsPerPage': 50}, 'items': [{'kind': 'youtube#searchResult', 'etag': 'AtCsHT1G8_iu3dLheZ4-WGEu6kI', 'id': {'kind': 'youtube#video', 'videoId': 'cy_6ZKurLS4'}, 'snippet': {'publishedAt': '2021-01-09T17:03:55Z', 'channelId': 'UCs4zWdCrlnXmpVkjOgAHs0w', 'title': '【DTM】PSYQUIさんとDTMや音楽について熱く語ります。', 'description': 'ばーちゃるとらっくめーかー。DTMや音楽の動画をあげてます。 たまにゲームや雑談なども。 アニメやゲーム、Vtuberさん等に楽曲提供してます。', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/cy_6ZKurLS4/default.jpg', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i.ytimg.com/vi/cy_6ZKurLS4/mqdefault.jpg', 'width': 320, 'height': 180}, 'high': {'url': 'https://i.ytimg.com/vi/cy_6ZKurLS4/hqdefault.jpg', 'width': 480, 'height': 360}}, 'channelTitle': 'ミディ / 作曲Vtuber', 'liveBroadcastContent': 'none', 'publishTime': '2021-01-09T17:03:55Z'}}, {'kind': 'youtube#searchResult'

How to retrieve individual items (each item is a youtube video from the search)

In [65]:
response['items'][1]

{'kind': 'youtube#searchResult',
 'etag': '9-mK4cAIpWSe6Hz0fNj8BP2FTZI',
 'id': {'kind': 'youtube#video', 'videoId': 'Jhq_44IotCM'},
 'snippet': {'publishedAt': '2019-09-09T17:47:12Z',
  'channelId': 'UCwobzUc3z-0PrFpoRxNszXQ',
  'title': 'Study Music, Work Music, Focus, Concentration, Meditation, Relaxing Music, Calm Music, Study,\xa0☯3601',
  'description': 'Study Music, Work Music, Focus, Concentration, Meditation, Relaxing Music, Calm Music, Study, ☯3601 - Are you looking for studying music, focus music or ...',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/Jhq_44IotCM/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/Jhq_44IotCM/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/Jhq_44IotCM/hqdefault.jpg',
    'width': 480,
    'height': 360}},
  'channelTitle': 'Yellow Brick Cinema - Relaxing Music',
  'liveBroadcastContent': 'none',
  'publishTime': '2019-09-09T17:47:12Z'}}

## Getting More Info Per Video (using the same API request as listing search results)  
Note after looking at stuff more: let's just use this API to get the list of video ID's, and then use the next one (`videos().list`) to get the actual info (because there's way more info available through that one, and I think it would be easier to do it all there)

Getting individual info on each item

How we get the <font color="red">youtube URL</font>

In [66]:
response['items'][1]['id']['videoId']

'Jhq_44IotCM'

In [67]:
print('https://www.youtube.com/watch?v='+response['items'][1]['id']['videoId'])

https://www.youtube.com/watch?v=Jhq_44IotCM


How to iterate through all the items **on the first page** 
(figure out how to iterate through pages next)  
<font color='red'>*Todo: edit this so it goes through more than just the first page*</font>

In [45]:
list_of_ids = []

for item in response['items']:
    list_of_ids.append(item['id']['videoId'])

list_of_ids

['hVUrtS77kgM',
 '5n9FWGUm2pc',
 'gMZhRJ5354o',
 '18zTOaTENLA',
 'Xs_HdCFbhJI',
 'Bwb5D-ILhAQ',
 'Hr_Oe4kQgYE',
 'eDJ8wSdleRw',
 'c-EkFgViaPE',
 'Nu56cTqohlM',
 'N11sB7ggzdw',
 'Tc-XxqEyolU',
 '_8dh_MzO_nU',
 'bZnR9QwbbxI',
 '2nLzFxeZ9Zo',
 '08wKGQRkg_U',
 '944IATvLj-4',
 '3L65PG_eZFg',
 'F8gPLTk_ehQ',
 'EeVaNmytazM',
 'rCwSmEZ1Jyc',
 'LWn28sKDWXo',
 'Dhu3VSj-qnk',
 '4zF2hldTyfs',
 'sQFo1Ekmnv8',
 'WbBLWOwNRmg',
 'c-La-lVYPro',
 'kndYAscsEDM',
 'yG30yDhpNps',
 '0DymLlXPq9c',
 'mvpw-E6IU_c',
 'snfo9Pq_HOE',
 'Gp0sqq1Uh0w',
 'WSNFVE3MUdc',
 'JxHc0dp2zDI',
 'WRDWRJMAsAo',
 'hQ2mBuU1Q3M',
 'EbV3boFOMII',
 'fBRBnSvbWQ8',
 'oIMRZU2w83w',
 'MqoCXM9xsfA',
 'BVBdqkHNl54',
 'DMiQw0BLMnY',
 'vQQ8fsbb-lg',
 'kKfBb8zUM3U',
 'IRD7WylAfkw',
 'tRuM4zD48AA',
 'lvMowQI0fHI',
 'iqfS8C8awlY',
 'DqNvIW-us-c']

To get the next page:

In [52]:
last_request = request
last_response = response

In [53]:
request = youtube_service.search().list_next(
                                        previous_request = last_request,
                                        previous_response = last_response)

In [54]:
response = request.execute()

In [60]:
for item in response['items']:
    print(item['snippet']['title'])
    print()

['study music', 'concentration music', 'focus music', 'study', 'relaxing music', 'studying music', 'focus', 'stress relief music', 'calm music', 'meditation music', 'alpha waves', 'background music', 'music for studying', 'brain power', 'calm music for studying', 'binaural beats', 'meditation', 'concentration', 'stress relief', 'work music', 'homework', 'intelligence', 'quarantine', 'memory', 'mindfulness', 'music for memory', 'relax', 'music for study work', 'music to study', 'zen music', 'yoga', 'yoga music', 'reading music', 'yellow brick cinema']



## More Info on Videos (new API request)

part can contain: contentDetails, fileDetails, id, liveStreamingDetails, localizations, player, processingDetails, recordingDetails, snippet, statistics, status, suggestions, topicDetails

See info here: https://developers.google.com/youtube/v3/docs/videos

<font color='red'>*Todo: look at what data here may be useful to save*</font>

In [46]:
request = youtube_service.videos().list(
            part='snippet',
            id=list_of_ids)

In [47]:
response = request.execute()

In [48]:
print(response)

{'kind': 'youtube#videoListResponse', 'etag': 'EQhjsKgPA7bzUEtMvxyJfy2mEnU', 'items': [{'kind': 'youtube#video', 'etag': '8uD89cCnitEIL9WCaqvfm6bFBk8', 'id': 'hVUrtS77kgM', 'snippet': {'publishedAt': '2021-03-09T17:16:00Z', 'channelId': 'UCMOgdURr7d8pOVlc-alkfRg', 'title': '🎵  Progressive House/Trance Mix 2021 🎵', 'description': "👕 OFFICIAL XKITO MERCH STORE: http://dbh.la/xkito\n💚 Official xKito Music Spotify Playlist: https://spoti.fi/3c43dZN\n\n\n👉 Tracklist:\n00:00 - 02:54 Tritonal ft. Angel Taylor - Getaway (Axis Remix)\n02:54 - 07:12 Adrian Alexander & Thomas Mengel - Timewalker\n07:12 - 10:43 Thomas Hayes – Cirrus (Adrian Alexander Remix)\n10:43 - 15:22 Farius - Initio (Farius Club Mix)\n15:22 - 18:40 Kapera - Feelin You (feat. Lauren L'aimant)\n18:40 - 21:56 Paul Arcane & Scolario - Voices\n21:56 - 25:55 Rodrigo Deem - You Said It All (feat. KARRA)\n25:55 - 29:50 Koyah & Deakin XD - All I Wanted\n29:50 - 34:36 Warsongs - Piercing Light (Mako Remix)\n\n\n⭐️ xKito Music:\nhttps:/

In [49]:
response['items'][0]

{'kind': 'youtube#video',
 'etag': '8uD89cCnitEIL9WCaqvfm6bFBk8',
 'id': 'hVUrtS77kgM',
 'snippet': {'publishedAt': '2021-03-09T17:16:00Z',
  'channelId': 'UCMOgdURr7d8pOVlc-alkfRg',
  'title': '🎵  Progressive House/Trance Mix 2021 🎵',
  'description': "👕 OFFICIAL XKITO MERCH STORE: http://dbh.la/xkito\n💚 Official xKito Music Spotify Playlist: https://spoti.fi/3c43dZN\n\n\n👉 Tracklist:\n00:00 - 02:54 Tritonal ft. Angel Taylor - Getaway (Axis Remix)\n02:54 - 07:12 Adrian Alexander & Thomas Mengel - Timewalker\n07:12 - 10:43 Thomas Hayes – Cirrus (Adrian Alexander Remix)\n10:43 - 15:22 Farius - Initio (Farius Club Mix)\n15:22 - 18:40 Kapera - Feelin You (feat. Lauren L'aimant)\n18:40 - 21:56 Paul Arcane & Scolario - Voices\n21:56 - 25:55 Rodrigo Deem - You Said It All (feat. KARRA)\n25:55 - 29:50 Koyah & Deakin XD - All I Wanted\n29:50 - 34:36 Warsongs - Piercing Light (Mako Remix)\n\n\n⭐️ xKito Music:\nhttps://www.facebook.com/xKitoMusic\nhttps://twitter.com/xkito_music\nhttps://instagr

In [51]:
response['items'][0]['snippet']

{'publishedAt': '2021-03-09T17:16:00Z',
 'channelId': 'UCMOgdURr7d8pOVlc-alkfRg',
 'title': '🎵  Progressive House/Trance Mix 2021 🎵',
 'description': "👕 OFFICIAL XKITO MERCH STORE: http://dbh.la/xkito\n💚 Official xKito Music Spotify Playlist: https://spoti.fi/3c43dZN\n\n\n👉 Tracklist:\n00:00 - 02:54 Tritonal ft. Angel Taylor - Getaway (Axis Remix)\n02:54 - 07:12 Adrian Alexander & Thomas Mengel - Timewalker\n07:12 - 10:43 Thomas Hayes – Cirrus (Adrian Alexander Remix)\n10:43 - 15:22 Farius - Initio (Farius Club Mix)\n15:22 - 18:40 Kapera - Feelin You (feat. Lauren L'aimant)\n18:40 - 21:56 Paul Arcane & Scolario - Voices\n21:56 - 25:55 Rodrigo Deem - You Said It All (feat. KARRA)\n25:55 - 29:50 Koyah & Deakin XD - All I Wanted\n29:50 - 34:36 Warsongs - Piercing Light (Mako Remix)\n\n\n⭐️ xKito Music:\nhttps://www.facebook.com/xKitoMusic\nhttps://twitter.com/xkito_music\nhttps://instagram.com/xkito.music/\nhttps://soundcloud.com/xkitomusic\nhttps://www.youtube.com/c/xKitoMusic\n\n\n📄 Sub

In [52]:
response['items'][0]['snippet']['title']

'🎵  Progressive House/Trance Mix 2021 🎵'

In [53]:
response['items'][0]['snippet']['description']

"👕 OFFICIAL XKITO MERCH STORE: http://dbh.la/xkito\n💚 Official xKito Music Spotify Playlist: https://spoti.fi/3c43dZN\n\n\n👉 Tracklist:\n00:00 - 02:54 Tritonal ft. Angel Taylor - Getaway (Axis Remix)\n02:54 - 07:12 Adrian Alexander & Thomas Mengel - Timewalker\n07:12 - 10:43 Thomas Hayes – Cirrus (Adrian Alexander Remix)\n10:43 - 15:22 Farius - Initio (Farius Club Mix)\n15:22 - 18:40 Kapera - Feelin You (feat. Lauren L'aimant)\n18:40 - 21:56 Paul Arcane & Scolario - Voices\n21:56 - 25:55 Rodrigo Deem - You Said It All (feat. KARRA)\n25:55 - 29:50 Koyah & Deakin XD - All I Wanted\n29:50 - 34:36 Warsongs - Piercing Light (Mako Remix)\n\n\n⭐️ xKito Music:\nhttps://www.facebook.com/xKitoMusic\nhttps://twitter.com/xkito_music\nhttps://instagram.com/xkito.music/\nhttps://soundcloud.com/xkitomusic\nhttps://www.youtube.com/c/xKitoMusic\n\n\n📄 Submit your music or art to xKito:\nYou can now submit your music/art directly to xKito through the submission platform! You can also submit your unrelea

In [54]:
response['items'][0]['snippet']['tags']

['electro',
 'electronic',
 'dance',
 'music',
 'EDM',
 'xkito',
 'xkitomusic',
 'anime',
 'animation',
 'kito',
 'gif',
 'prog house',
 'prog',
 'progressive',
 'house',
 'trance',
 'progressive house',
 'progressive trance',
 'mix',
 'compilation',
 'playlist']

<hr>

In [94]:
# Allows us to connect to the YouTube API service
from googleapiclient.discovery import build
import math

In [57]:
API_KEY = ""

# Read in the API key
with open("../YouTube API Key.txt") as f:
    for line in f:
        API_KEY = line

# Create the service
youtube_service = build('youtube', 'v3', developerKey=API_KEY)

In [55]:
keywords = ['xkito', 'electric swing','reol','animal crossing']

In [104]:
def search_videos(search_terms, pages=5, results_per_page = 50):
    """
    Search videos on YouTube given a list of various keywords or key phrases.
    Return a list of YouTube video IDs.
    
    search_terms: a list of search terms. each search term will be a new search
    pages: the number of pages we want to retrieve per search (default 5)
    results_per_page: the number of results per page we want to retrieve per search (max is 50)
    returns a list of video ids
    
    """
    
    # initialize empty list to store video ids
    list_of_ids = []
    
    # ID for the video category of music is 10
    MUSIC_CATEGORY_ID = 10

    for search_term in search_terms:
        
        # get the first page of videos for the search term
        request = youtube_service.search().list(
                    part='snippet',
                    maxResults=results_per_page,
                    q=search_term,
                    type='video',
                    videoCategoryId = MUSIC_CATEGORY_ID,
                    )
        response = request.execute()

        # iterate through the response and save just the video id's
        for item in response['items']:
            list_of_ids.append(item['id']['videoId'])
                
        # we already searched once, so get the remaining X-1 pages (where X is number of pages)
        for page in range(1, pages):
            last_request = request
            last_response = response

            request = youtube_service.search().list_next(
                                    previous_request = last_request,
                                    previous_response = last_response)
            response = request.execute()
            
            # iterate through the response and save just the video id's
            for item in response['items']:
                list_of_ids.append(item['id']['videoId'])

    # done!    
    return list_of_ids

In [108]:
ids = search_videos(keywords)

In [ ]:
def get_video_data(video_ids):
    """
    Saves a csv (for now) of video data.
    
    video_ids: the list of video ids
    """
    
    
    
    

Note: can only do 50 at a time?

In [145]:
math.ceil(len(ids)/50)

20

In [146]:
len(ids)

1000

In [147]:
request = youtube_service.videos().list(
            part='snippet, contentDetails, statistics, topicDetails',
            id=ids[950:])

response = request.execute()

In [128]:
response['items'][0]

{'kind': 'youtube#video',
 'etag': 'GNS319mc0jP8MQV9Nm_mipul6EA',
 'id': 'DqNvIW-us-c',
 'snippet': {'publishedAt': '2017-11-13T19:15:22Z',
  'channelId': 'UCMOgdURr7d8pOVlc-alkfRg',
  'title': 'MARNIK - Burn (feat. Rookies)',
  'description': '♫ Buy/Stream: https://marnik.lnk.to/burn\n\n\n[Genre: Electronic/Pop]\n\n[Label: UMG Recordings]\n\n\n►Marnik:\nhttps://soundcloud.com/marnikofficial\nhttps://www.facebook.com/Marnikofficial/\nhttps://twitter.com/marnikofficial\nhttps://www.instagram.com/marnikofficial/\nhttps://www.youtube.com/marnikofficial\n\n\n►Rookies:\nhttps://soundcloud.com/r0okies\nhttps://twitter.com/r0okies\nhttps://www.instagram.com/r0okies/\nhttps://www.facebook.com/r0okies\nhttps://twitter.com/r0okies\nhttps://www.youtube.com/channel/UCI0OcrtsHQbGWIlKNjllX7Q\n\n\n►xKito Music:\nhttps://www.facebook.com/xKitoMusic\nhttps://twitter.com/xkito_music\nhttps://instagram.com/belgin.xk/\nhttps://soundcloud.com/xkitomusic\nhttps://www.youtube.com/c/xKitoMusic\n\n\n►Submit yo

In [148]:
response['items'][0]['snippet']['title']

'Prologue (Campfire) - Animal Crossing: New Horizons Music Extended [OST]'

In [149]:
response['items'][0]['snippet']['description']

'Animal Crossing: New Horizons music that has been extended to play for at least 15.5 minutes.\n\nComposer(s): Yasuaki Iwata, Yumi Takahashi, Shinobu Nagata, Sayako Doi, Masato Ohashi\nArranger(s): Yasuaki Iwata, Yumi Takahashi, Shinobu Nagata, Sayako Doi, Masato Ohashi\n\nDeveloper(s): Nintendo, Nintendo Entertainment Planning & Development\nPublisher(s): Nintendo\n\nFull Playlist: https://www.youtube.com/playlist?list=PLsz7MwL4fp2LjGFdIus90HPfI6BYINSx8\n\nPlease Subscribe and Like to Keep this Channel Going, It Genuinely Helps.\n\nSocial Media: https://mobile.twitter.com/cellextyn'

In [161]:
try:
    print(response['items'][0]['snippet']['tags'])
except:
    print("No tags")

No tags


In [151]:
response['items'][0]['snippet']['channelTitle']

'Cellextyn X'

In [152]:
response['items'][0]['snippet']['localized']['title']

'Prologue (Campfire) - Animal Crossing: New Horizons Music Extended [OST]'

In [153]:
response['items'][0]['snippet']['localized']['description']

'Animal Crossing: New Horizons music that has been extended to play for at least 15.5 minutes.\n\nComposer(s): Yasuaki Iwata, Yumi Takahashi, Shinobu Nagata, Sayako Doi, Masato Ohashi\nArranger(s): Yasuaki Iwata, Yumi Takahashi, Shinobu Nagata, Sayako Doi, Masato Ohashi\n\nDeveloper(s): Nintendo, Nintendo Entertainment Planning & Development\nPublisher(s): Nintendo\n\nFull Playlist: https://www.youtube.com/playlist?list=PLsz7MwL4fp2LjGFdIus90HPfI6BYINSx8\n\nPlease Subscribe and Like to Keep this Channel Going, It Genuinely Helps.\n\nSocial Media: https://mobile.twitter.com/cellextyn'

In [154]:
response['items'][0]['contentDetails']['duration']

'PT30M2S'

Come back to this later maybe

In [155]:
response['items'][0]['contentDetails']['contentRating']

{}

In [156]:
response['items'][40]['statistics']

{'viewCount': '130022',
 'likeCount': '13984',
 'dislikeCount': '12',
 'favoriteCount': '0',
 'commentCount': '574'}

In [157]:
response['items'][30]['topicDetails']

{'topicCategories': ['https://en.wikipedia.org/wiki/Music']}